In [56]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

In [115]:
DATAFOLDER = "data"

def RenameDimensionsAndVariables(dataset, dims_dict = dict(), vars_dict = False) :
    try:
        # rename the dimensions
        dataset = dataset.rename_dims(dims_dict)
        # rename coords assosiatet with dimensions
        try :
            dataset = dataset.rename_vars(dims_dict)
        except Exception as E :
            print(E)
        # rename the variables if necessary
        if vars_dict :
            dataset = dataset.rename_vars(vars_dict)
        return dataset
    except Exception as E :
        print(E)
        return None

# data from the Hydrographic Groups 1+2
hydrographic_mean = xr.open_dataset(DATAFOLDER + "/group1_mean_transport.nc")
hydrographic_mean = RenameDimensionsAndVariables(
            dataset = hydrographic_mean,
            dims_dict = dict(
                LONGITUDE = "longitude",
                PRESSURE = "pressure",
                ),
            vars_dict = dict(
                __xarray_dataarray_variable__ = "hydrographic_mean"
                ))
hydrographic_climatology = xr.open_dataset(DATAFOLDER + "/group1_climatology_transport.nc")
hydrographic_climatology = RenameDimensionsAndVariables(
            dataset = hydrographic_climatology,
            dims_dict = dict(
                LONGITUDE = "longitude",
                PRESSURE = "pressure",
                TIME = "month"
                ),
            vars_dict = dict(
                __xarray_dataarray_variable__ = "hydrographic_climatology"
                ))
hydrographic_mean.merge(hydrographic_climatology)
hydrographic_variability = xr.open_dataset(DATAFOLDER + "/group1_mean_transport.nc")

# western_boundary = xr.open_dataset(DATAFOLDER + "/group3_western_boundary_transport.nc")
# ekman_transport = xr.open_dataset(DATAFOLDER + "/group5_ekmann_transport.nc")
# data = [
#     hydrographic_mean,
#     hydrographic_climatology,
#     hydrographic_variability,
#     western_boundary,
#     ekman_transport,
# ]
# create a uniform data_set with only time as dimension and monthly steps
# hydrographic_mean

<xarray.Dataset>
Dimensions:                   (longitude: 97, month: 12, pressure: 58)
Coordinates:
  * pressure                  (pressure) float32 2.5 10.0 20.0 ... 1900.0 1975.0
  * longitude                 (longitude) float32 -35.75 -35.25 ... 11.75 12.25
  * month                     (month) float32 0.5 1.5 2.5 3.5 ... 9.5 10.5 11.5
Data variables:
    hydrographic_mean         (pressure, longitude) float64 ...
    hydrographic_climatology  (month, pressure, longitude) float64 ...

<h3> Calculat 

In [77]:
data = xr.Dataset(
)
time = pd.date_range(

        "01/2000",

        periods=20*12,

        freq=pd.DateOffset(months=1),

    )
time

DatetimeIndex(['2000-01-01', '2000-02-01', '2000-03-01', '2000-04-01',
               '2000-05-01', '2000-06-01', '2000-07-01', '2000-08-01',
               '2000-09-01', '2000-10-01',
               ...
               '2019-03-01', '2019-04-01', '2019-05-01', '2019-06-01',
               '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
               '2019-11-01', '2019-12-01'],
              dtype='datetime64[ns]', length=240, freq='<DateOffset: months=1>')

In [85]:
ds = xr.Dataset(

    data_vars=dict(

        # time2=(["time"], time),

    ),

    coords=dict(

        time=(["time"],time),

    ),

    attrs=dict(description="Weather related data."),

)
ds["ekman_transport"] = (1e-6*ekman_transport.volume_transport_y).where(ekman_transport.atlantic_mask).mean("latitude").sum('longitude')
ds

<xarray.Dataset>
Dimensions:          (time: 240)
Coordinates:
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2019-12-01
Data variables:
    ekman_transport  (time) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    description:  Weather related data.

In [50]:
hydrographic_mean

<xarray.Dataset>
Dimensions:                        (LONGITUDE: 97, PRESSURE: 58)
Coordinates:
  * PRESSURE                       (PRESSURE) float32 2.5 10.0 ... 1900.0 1975.0
  * LONGITUDE                      (LONGITUDE) float32 -35.75 -35.25 ... 12.25
Data variables:
    __xarray_dataarray_variable__  (PRESSURE, LONGITUDE) float64 ...

In [65]:
western_boundary.resample(time="1MS").mean(dim="time")
# western_boundary.resample(time="1MS").std(dim="time").Transport_pos.plot()
# hydrographic_variability.integrated_transport.resample(time="1MS").mean(dim="time")
# (1e-6*ekman_transport.volume_transport_y).where(ekman_transport.atlantic_mask).mean("latitude").sum('longitude').plot()

<xarray.Dataset>
Dimensions:             (time: 76)
Coordinates:
  * time                (time) datetime64[ns] 2013-07-01 ... 2019-10-01
Data variables:
    Transport_pos       (time) float64 18.81 22.19 17.67 ... 21.72 22.49 22.12
    Transport_neg       (time) float64 -5.939 -12.61 -9.988 ... -7.164 -11.54
    Transport_pos_mean  (time) float64 24.6 24.6 24.6 24.6 ... 24.6 24.6 24.6
    Transport_neg_mean  (time) float64 -8.8 -8.8 -8.8 -8.8 ... -8.8 -8.8 -8.8